<a href="https://colab.research.google.com/github/SeungukJeong/CUAI_HandsOn-ML-Study-T1_2022/blob/main/INGV_1213_Pycaret_basic_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Optuna: 하이퍼파라미터 최적화 프레임워크 <br>
https://ssoonidev.tistory.com/107 <br><br> 




#### import package

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Dec 13 16:23:15 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    53W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Install PyCaret

In [4]:
!pip install --use-deprecated=legacy-resolver pycaret[full] --quiet
!pip install numpy==1.20 --quiet
# !pip install pycaret --quiet 



# install full-pycaret for xgboost 
# https://stackoverflow.com/questions/68477475/problem-on-installing-pycaret-full-on-google-colab 

# "Estimator xgboost not available" for PyCaret in google colab?
# https://stackoverflow.com/questions/67068965/estimator-xgboost-not-available-for-pycaret-in-google-colab 

In [5]:
from pycaret.regression import * 

In [6]:
import os
project_path = '/content/drive/MyDrive/FinalProject'

os.chdir(project_path)
os.getcwd()

'/content/drive/MyDrive/ML4AI_22Fall/FinalProject'

In [7]:
import pandas as pd
# import glob
import math 

from tqdm.notebook import tqdm
from tqdm import tqdm
import time

Each file contains **`10 minutes of logs`** from **`10 different sensors`** arrayed around a volcano. <br><br>
The readings have been normalized within each segment <br> 
fall within the range of int16 values. 

In [8]:
import pandas as pd

In [9]:
# pd.set_option('display.max_columns', None)

In [10]:
train = pd.read_csv('train.csv')
basic_train = pd.read_csv('./Seunguks_Practice/basic_train_stat_add.csv')

# basic_train = pd.read_csv('./Feature_data/df_merge.csv')
sample_submission = pd.read_csv("sample_submission.csv")

In [11]:
basic_train.head(3)

,sensor_1_mean,sensor_1_std,sensor_1_min,sensor_1_25%,sensor_1_50%,sensor_1_75%,sensor_1_max,sensor_1_skew,sensor_1_mad,sensor_1_kurtosis,sensor_2_mean,sensor_2_std,sensor_2_min,sensor_2_25%,sensor_2_50%,sensor_2_75%,sensor_2_max,sensor_2_skew,sensor_2_mad,sensor_2_kurtosis,sensor_3_mean,sensor_3_std,sensor_3_min,sensor_3_25%,sensor_3_50%,sensor_3_75%,sensor_3_max,sensor_3_skew,sensor_3_mad,sensor_3_kurtosis,sensor_4_mean,sensor_4_std,sensor_4_min,sensor_4_25%,sensor_4_50%,sensor_4_75%,sensor_4_max,sensor_4_skew,sensor_4_mad,sensor_4_kurtosis,sensor_5_mean,sensor_5_std,sensor_5_min,sensor_5_25%,sensor_5_50%,sensor_5_75%,sensor_5_max,sensor_5_skew,sensor_5_mad,sensor_5_kurtosis,sensor_6_mean,sensor_6_std,sensor_6_min,sensor_6_25%,sensor_6_50%,sensor_6_75%,sensor_6_max,sensor_6_skew,sensor_6_mad,sensor_6_kurtosis,sensor_7_mean,sensor_7_std,sensor_7_min,sensor_7_25%,sensor_7_50%,sensor_7_75%,sensor_7_max,sensor_7_skew,sensor_7_mad,sensor_7_kurtosis,sensor_8_mean,sensor_8_std,sensor_8_min,sensor_8_25%,sensor_8_50%,sensor_8_75%,sensor_8_max,sensor_8_skew,sensor_8_mad,sensor_8_kurtosis,sensor_9_mean,sensor_9_std,sensor_9_min,sensor_9_25%,sensor_9_50%,sensor_9_75%,sensor_9_max,sensor_9_skew,sensor_9_mad,sensor_9_kurtosis,sensor_10_mean,sensor_10_std,sensor_10_min,sensor_10_25%,sensor_10_50%,sensor_10_75%,sensor_10_max,sensor_10_skew,sensor_10_mad,sensor_10_kurtosis,time,segment_id
0,10.297245,1080.658741,-4329.0,-699.0,0.0,723.0,4866.0,0.021152,853.814395,0.202543,0.402560,1388.566153,-6228.0,-909.0,0.0,908.0,9097.0,0.028820,1093.020824,0.386810,-1.329711,1097.419061,-4042.0,-742.0,0.0,746.0,3833.0,-0.022526,878.357010,-0.070693,-2.898518,1076.602492,-4643.0,-723.0,0.0,723.0,4805.0,0.008261,856.129736,0.153629,0.705788,374.755707,-1511.0,-251.0,0.0,255.0,1487.0,-0.013522,299.213040,0.037729,15.375994,623.507169,-2458.0,-401.0,0.0,431.0,2340.0,0.029198,496.011514,0.059886,-16.563007,583.053755,-3453.0,-404.0,0.0,373.0,3027.0,-0.107006,460.383029,0.354395,-2.899902,717.680114,-6437.0,-471.0,0.0,480.0,5009.0,-0.050644,568.200315,0.455135,-9.972767,1301.715061,-4868.0,-887.0,0.0,852.0,4572.0,-0.005730,1024.379671,0.037657,2.216696,1697.782379,-6801.0,-1159.0,0.0,1149.0,7266.0,0.026519,1355.517274,0.068443,7169486,579683001
1,4.469659,804.238649,-3498.0,-532.0,0.0,552.0,3402.0,-0.005650,641.805415,0.052170,-7.146064,1081.021949,-5292.0,-697.0,0.0,685.0,5635.0,0.018737,843.357139,0.582011,1.894302,860.557291,-4885.0,-572.0,0.0,578.0,4286.0,-0.032397,684.339251,0.249335,-5.701472,823.601694,-3312.0,-543.0,0.0,546.0,3010.0,-0.053866,653.891062,0.089504,-1.740221,311.186658,-1609.0,-206.0,0.0,198.0,1515.0,0.007010,244.758124,0.364798,1.654339,570.718830,-2594.0,-378.0,0.0,387.0,2865.0,-0.006887,453.822345,0.139898,-6.575724,583.402283,-2749.0,-398.0,0.0,383.0,2469.0,-0.026995,464.164999,0.102562,1.547574,676.528623,-4059.0,-445.0,0.0,448.0,4983.0,-0.040716,533.362862,0.695588,-8.832286,929.990412,-3722.0,-614.0,0.0,604.0,3845.0,-0.022567,730.406283,0.228639,38.717871,1407.583604,-5006.0,-911.0,0.0,935.0,6596.0,0.171276,1113.111210,0.171414,8297866,580262567
2,-1.726988,480.404868,-3785.0,-275.0,0.0,274.0,5152.0,-0.039636,354.969467,3.272190,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,1.370927,261.396786,-1227.0,-173.0,0.0,174.0,1080.0,0.026708,207.416941,0.120075,-0.892818,359.476265,-1791.0,-231.0,0.0,235.0,1685.0,-0.034777,282.772764,0.312749,-0.157064,259.300180,-1774.0,-161.0,0.0,159.0,1467.0,-0.027120,198.857417,1.277905,-0.428393,472.856942,-2353.0,-311.0,0.0,318.0,2008.0,-0.050738,375.359632,0.101820,3.400160,478.710461,-2605.0,-313.0,0.0,319.0,3298.0,0.016186,378.711236,0.338472,-0.603707,612.779222,-9700.0,-373.0,0.0,367.0,8299.0,0.060368,458.073581,8.092038,0.038949,326.951763,-1409.0,-210.0,0.0,211.0,1781.0,-0.024783,256.624979,0.345410,-7.849603,713.271858,-3184.0,-463.0,0.0,455.0,3206.0,-0.027107,558.263159,0.410877,27527640,579307667


In [12]:
basic_train.set_index('segment_id')

,sensor_1_mean,sensor_1_std,sensor_1_min,sensor_1_25%,sensor_1_50%,sensor_1_75%,sensor_1_max,sensor_1_skew,sensor_1_mad,sensor_1_kurtosis,sensor_2_mean,sensor_2_std,sensor_2_min,sensor_2_25%,sensor_2_50%,sensor_2_75%,sensor_2_max,sensor_2_skew,sensor_2_mad,sensor_2_kurtosis,sensor_3_mean,sensor_3_std,sensor_3_min,sensor_3_25%,sensor_3_50%,sensor_3_75%,sensor_3_max,sensor_3_skew,sensor_3_mad,sensor_3_kurtosis,sensor_4_mean,sensor_4_std,sensor_4_min,sensor_4_25%,sensor_4_50%,sensor_4_75%,sensor_4_max,sensor_4_skew,sensor_4_mad,sensor_4_kurtosis,sensor_5_mean,sensor_5_std,sensor_5_min,sensor_5_25%,sensor_5_50%,sensor_5_75%,sensor_5_max,sensor_5_skew,sensor_5_mad,sensor_5_kurtosis,sensor_6_mean,sensor_6_std,sensor_6_min,sensor_6_25%,sensor_6_50%,sensor_6_75%,sensor_6_max,sensor_6_skew,sensor_6_mad,sensor_6_kurtosis,sensor_7_mean,sensor_7_std,sensor_7_min,sensor_7_25%,sensor_7_50%,sensor_7_75%,sensor_7_max,sensor_7_skew,sensor_7_mad,sensor_7_kurtosis,sensor_8_mean,sensor_8_std,sensor_8_min,sensor_8_25%,sensor_8_50%,sensor_8_75%,sensor_8_max,sensor_8_skew,sensor_8_mad,sensor_8_kurtosis,sensor_9_mean,sensor_9_std,sensor_9_min,sensor_9_25%,sensor_9_50%,sensor_9_75%,sensor_9_max,sensor_9_skew,sensor_9_mad,sensor_9_kurtosis,sensor_10_mean,sensor_10_std,sensor_10_min,sensor_10_25%,sensor_10_50%,sensor_10_75%,sensor_10_max,sensor_10_skew,sensor_10_mad,sensor_10_kurtosis,time
segment_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
579683001,10.297245,1080.658741,-4329.0,-699.0,0.0,723.0,4866.0,0.021152,853.814395,0.202543,0.402560,1388.566153,-6228.0,-909.0,0.0,908.0,9097.0,0.028820,1093.020824,0.386810,-1.329711,1097.419061,-4042.0,-742.0,0.0,746.0,3833.0,-0.022526,878.357010,-0.070693,-2.898518,1076.602492,-4643.0,-723.0,0.0,723.0,4805.0,0.008261,856.129736,0.153629,0.705788,374.755707,-1511.0,-251.0,0.0,255.0,1487.0,-0.013522,299.213040,0.037729,15.375994,623.507169,-2458.0,-401.0,0.0,431.0,2340.0,0.029198,496.011514,0.059886,-16.563007,583.053755,-3453.0,-404.0,0.0,373.0,3027.0,-0.107006,460.383029,0.354395,-2.899902,717.680114,-6437.0,-471.0,0.0,480.0,5009.0,-0.050644,568.200315,0.455135,-9.972767,1301.715061,-4868.0,-887.0,0.0,852.0,4572.0,-0.005730,1024.379671,0.037657,2.216696,1697.782379,-6801.0,-1159.0,0.0,1149.0,7266.0,0.026519,1355.517274,0.068443,7169486
580262567,4.469659,804.238649,-3498.0,-532.0,0.0,552.0,3402.0,-0.005650,641.805415,0.052170,-7.146064,1081.021949,-5292.0,-697.0,0.0,685.0,5635.0,0.018737,843.357139,0.582011,1.894302,860.557291,-4885.0,-572.0,0.0,578.0,4286.0,-0.032397,684.339251,0.249335,-5.701472,823.601694,-3312.0,-543.0,0.0,546.0,3010.0,-0.053866,653.891062,0.089504,-1.740221,311.186658,-1609.0,-206.0,0.0,198.0,1515.0,0.007010,244.758124,0.364798,1.654339,570.718830,-2594.0,-378.0,0.0,387.0,2865.0,-0.006887,453.822345,0.139898,-6.575724,583.402283,-2749.0,-398.0,0.0,383.0,2469.0,-0.026995,464.164999,0.102562,1.547574,676.528623,-4059.0,-445.0,0.0,448.0,4983.0,-0.040716,533.362862,0.695588,-8.832286,929.990412,-3722.0,-614.0,0.0,604.0,3845.0,-0.022567,730.406283,0.228639,38.717871,1407.583604,-5006.0,-911.0,0.0,935.0,6596.0,0.171276,1113.111210,0.171414,8297866
579307667,-1.726988,480.404868,-3785.0,-275.0,0.0,274.0,5152.0,-0.039636,354.969467,3.272190,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,1.370927,261.396786,-1227.0,-173.0,0.0,174.0,1080.0,0.026708,207.416941,0.120075,-0.892818,359.476265,-1791.0,-231.0,0.0,235.0,1685.0,-0.034777,282.772764,0.312749,-0.157064,259.300180,-1774.0,-161.0,0.0,159.0,1467.0,-0.027120,198.857417,1.277905,-0.428393,472.856942,-2353.0,-311.0,0.0,318.0,2008.0,-0.050738,375.359632,0.101820,3.400160,478.710461,-2605.0,-313.0,0.0,319.0,3298.0,0.016186,378.711236,0.338472,-0.603707,612.779222,-9700.0,-373.0,0.0,367.0,8299.0,0.060368,458.073581,8.092038,0.038949,326.951763,-1409.0,-210.0,0.0,211.0,1781.0,-0.024783,256.624979,0.345410,-7.849603,713.271858,-3184.0,-463.0,0.0,455.0,3206.0,-0.027107,558.26315

# PyCaret

In [13]:
# !pip install pycaret==2.1.2
# !pip install scikit-learn==0.23 # 반드시 Scikit-learn 버전을 맞춰줘야 합니다!
# !pip install scipy==1.5.3 # 반드시 Scipy 버전을 맞춰줘야 합니다! 
# !pip install pyyaml==5.4.1

In [14]:
# from pycaret.regression import * 

In [15]:
model = setup(data = basic_train,
              target = 'time',
              train_size = 0.7,
              session_id = 123,
              silent = True)

# et1 = create_model('et')
# final_et1 = finalize_model(et1)


# xt_clu = setup(X_new1, target = 'time_to_eruption',session_id=17,silent = True)
# et1 = create_model('et')
# final_et1 = finalize_model(et1)

,Description,Value
0,session_id,123
1,Target,time
2,Original Data,"(4431, 102)"
3,Missing Values,False
4,Numeric Features,101
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(3101, 87)"


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=False, features_todrop=[],
                                      id_columns=[], ml_usecase='regression',
                                      numerical_features=[], target='time',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strategy='...
                ('scaling', 'passthrough'), ('P_transform', 'passthrough'),
                ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                ('cluster_all', 'passthrough'),
              

In [16]:
# # 직접 모델 종류 지정하여 생성
# xgb = create_model('xgboost') # creates a model and scores it using stratified cross validation (가능한 모델estimator 리스트는 docstring 참고)

In [17]:
# xgb_tuned = tune_model(xgb, optimize='MAE') # tunes the hyperparameters of a model on a pre-defined search space and scores it using stratified cross validation

In [18]:
top_5_models = compare_models(sort = 'MAE',
                              n_select = 5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,4.182394e+06,3.629231e+13,6.011348e+06,0.8001,0.6098,1.4629,0.538
catboost,CatBoost Regressor,4.790072e+06,4.128999e+13,6.412971e+06,0.7724,0.6483,1.7274,11.034
lightgbm,Light Gradient Boosting Machine,5.123899e+06,4.851624e+13,6.954849e+06,0.7325,0.6666,1.7726,0.293
rf,Random Forest Regressor,5.134289e+06,5.025482e+13,7.078770e+06,0.7227,0.6778,1.7139,1.793
xgboost,Extreme Gradient Boosting,5.305122e+06,5.151071e+13,7.167717e+06,0.7156,0.6755,1.5829,0.758
dt,Decision Tree Regressor,5.892854e+06,1.086631e+14,1.040863e+07,0.3994,0.8650,1.2970,0.053
gbr,Gradient Boosting Regressor,6.762059e+06,7.254884e+13,8.513269e+06,0.5998,0.7764,2.5621,0.844
ada,AdaBoost Regressor,9.869188e+06,1.281605e+14,1.131812e+07,0.2930,0.9555,3.5964,0.329
lasso,Lasso Regression,1.042830e+07,1.662718e+14,1.283287e+07,0.0840,0.9639,4.5066,0.155
br,Bayesian Ridge,1.043999e+07,1.534533e+14,1.238520e+07,0.1532,0.9733,4.4218,0.027


INFO:logs:create_model_container: 19
INFO:logs:master_model_container: 19
INFO:logs:display_container: 2
INFO:logs:[ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=100, n_jobs=-1, oob_score=False,
                    random_state=123, verbose=0, warm_start=False), <catboost.core.CatBoostRegressor object at 0x7f948582d640>, LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=123, reg_alp

In [19]:
top_5_models

[ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     max_samples=None, min_impurity_decrease=0.0,
                     min_impurity_split=None, min_samples_leaf=1,
                     min_samples_split=2, min_weight_fraction_leaf=0.0,
                     n_estimators=100, n_jobs=-1, oob_score=False,
                     random_state=123, verbose=0, warm_start=False),
 LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=123, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
 RandomForestRegressor(bootstrap=True, ccp_alpha=0

In [20]:
evaluate_model(top_5_models[0])

INFO:logs:Initializing evaluate_model()
INFO:logs:evaluate_model(estimator=ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=100, n_jobs=-1, oob_score=False,
                    random_state=123, verbose=0, warm_start=False), fold=None, fit_kwargs=None, plot_kwargs=None, feature_name=None, groups=None, use_train_data=False)


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [21]:
predict_model(top_5_models[0]) # 미리 제외시켜두었던 test data에 대한 예측 결과값

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=100, n_jobs=-1, oob_score=False,
                    random_state=123, verbose=0, warm_start=False), probability_threshold=None, encoded_labels=True, drift_report=False, raw_score=False, round=4, verbose=True, ml_usecase=MLUsecase.REGRESSION, display=None, drift_kwargs=None)
INFO:logs:Checking exceptions
INFO:logs:Preloading libraries
INFO:logs:Preparing display monitor


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extra Trees Regressor,3.956688e+06,3.342370e+13,5.781323e+06,0.8149,0.5948,1.1765


,sensor_1_mean,sensor_1_std,sensor_1_min,sensor_1_50%,sensor_1_75%,sensor_1_max,sensor_1_skew,sensor_1_kurtosis,sensor_2_mean,sensor_2_std,...,sensor_10_std,sensor_10_min,sensor_10_25%,sensor_10_50%,sensor_10_max,sensor_10_skew,sensor_10_kurtosis,segment_id,time,Label
0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,-1.531141,982.825989,...,554.055237,-3098.0,-348.0,0.0,3592.0,0.043389,0.878872,5.949085e+08,47775708,46877282.19
1,2.774204,271.149200,-1487.0,0.0,168.0,1732.0,0.043931,1.781223,0.000000,0.000000,...,588.235718,-4149.0,-367.0,0.0,4052.0,0.052804,1.859576,4.786741e+08,17722981,17921559.36
2,-3.869586,155.141571,-656.0,0.0,98.0,748.0,-0.086593,0.321308,-1.642989,464.430237,...,291.042755,-1556.0,-184.0,0.0,1545.0,0.031023,0.812098,1.759210e+09,38195716,33208399.48
3,1.258896,507.877563,-4190.0,0.0,303.0,3769.0,0.000385,2.031903,5.067732,826.030701,...,564.066833,-2553.0,-364.0,0.0,2584.0,0.013483,0.312417,2.889141e+08,43673535,32861707.31
4,-0.967134,369.767181,-1609.0,0.0,229.0,1919.0,0.013579,0.864417,-1.387227,930.861328,...,903.474915,-4721.0,-561.0,0.0,5130.0,-0.022145,1.884141,1.534910e+09,84390,13513516.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1325,7.926568,378.016266,-1945.0,0.0,256.0,1636.0,0.001878,0.621614,0.015383,565.777527,...,776.656616,-4215.0,-482.0,0.0,5889.0,0.076811,1.427138,1.357148e+09,47055965,30424081.11
1326,0.750104,645.540283,-3164.0,0.0,417.0,2989.0,0.015595,0.574703,5.097498,904.946106,...,1003.427673,-5722.0,-642.0,0.0,5553.0,-0.009897,0.845515,2.031745e+09,15487372,18038599.95
1327,0.805003,603.870667,-2374.0,0.0,398.0,2501.0,0.018574,0.043082,4.141681,1787.828979,...,1237.712891,-5228.0,-812.0,0.0,5227.0,-0.016879,0.192274,1.070819e+08,428461,4114236.32
1328,1.021250,754.624329,-7348.0,0.0,386.0,6818.0,0.011249,6.800555,0.553224,756.680054,...,744.890503,-3657.0,-481.0,0.0,4037.0,-0.015176,0.576139,5.712796e+08,33820073,29251045.80


In [22]:
blended = blend_models(estimator_list = top_5_models,
                       fold = 10,
                       optimize = 'MAE')


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,5.031873e+06,4.718479e+13,6.869119e+06,0.7441,0.7047,2.0718
1,4.409320e+06,3.479677e+13,5.898879e+06,0.8006,0.5843,0.8005
2,4.827044e+06,4.316930e+13,6.570335e+06,0.7696,0.6537,0.9578
3,5.058823e+06,4.975531e+13,7.053745e+06,0.7358,0.6891,1.9386
4,4.405920e+06,3.946465e+13,6.282089e+06,0.7849,0.6197,2.2423
5,4.740003e+06,4.364575e+13,6.606493e+06,0.7411,0.6185,0.7733
6,4.637582e+06,3.842224e+13,6.198568e+06,0.7970,0.7186,4.0342
7,4.657697e+06,4.046598e+13,6.361287e+06,0.7695,0.6564,1.3766
8,4.531765e+06,3.575752e+13,5.979759e+06,0.8008,0.4761,0.4742


INFO:logs:create_model_container: 20
INFO:logs:master_model_container: 20
INFO:logs:display_container: 4
INFO:logs:VotingRegressor(estimators=[('et',
                             ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0,
                                                 criterion='mse',
                                                 max_depth=None,
                                                 max_features='auto',
                                                 max_leaf_nodes=None,
                                                 max_samples=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=1

In [23]:
predict_model(blended)

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=VotingRegressor(estimators=[('et',
                             ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0,
                                                 criterion='mse',
                                                 max_depth=None,
                                                 max_features='auto',
                                                 max_leaf_nodes=None,
                                                 max_samples=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100, n_jobs=-1,
                           

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,4.395136e+06,3.747743e+13,6.121881e+06,0.7924,0.6368,1.3439


,sensor_1_mean,sensor_1_std,sensor_1_min,sensor_1_50%,sensor_1_75%,sensor_1_max,sensor_1_skew,sensor_1_kurtosis,sensor_2_mean,sensor_2_std,...,sensor_10_std,sensor_10_min,sensor_10_25%,sensor_10_50%,sensor_10_max,sensor_10_skew,sensor_10_kurtosis,segment_id,time,Label
0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,-1.531141,982.825989,...,554.055237,-3098.0,-348.0,0.0,3592.0,0.043389,0.878872,5.949085e+08,47775708,4.442317e+07
1,2.774204,271.149200,-1487.0,0.0,168.0,1732.0,0.043931,1.781223,0.000000,0.000000,...,588.235718,-4149.0,-367.0,0.0,4052.0,0.052804,1.859576,4.786741e+08,17722981,1.652261e+07
2,-3.869586,155.141571,-656.0,0.0,98.0,748.0,-0.086593,0.321308,-1.642989,464.430237,...,291.042755,-1556.0,-184.0,0.0,1545.0,0.031023,0.812098,1.759210e+09,38195716,3.516601e+07
3,1.258896,507.877563,-4190.0,0.0,303.0,3769.0,0.000385,2.031903,5.067732,826.030701,...,564.066833,-2553.0,-364.0,0.0,2584.0,0.013483,0.312417,2.889141e+08,43673535,3.272821e+07
4,-0.967134,369.767181,-1609.0,0.0,229.0,1919.0,0.013579,0.864417,-1.387227,930.861328,...,903.474915,-4721.0,-561.0,0.0,5130.0,-0.022145,1.884141,1.534910e+09,84390,1.488950e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1325,7.926568,378.016266,-1945.0,0.0,256.0,1636.0,0.001878,0.621614,0.015383,565.777527,...,776.656616,-4215.0,-482.0,0.0,5889.0,0.076811,1.427138,1.357148e+09,47055965,2.944545e+07
1326,0.750104,645.540283,-3164.0,0.0,417.0,2989.0,0.015595,0.574703,5.097498,904.946106,...,1003.427673,-5722.0,-642.0,0.0,5553.0,-0.009897,0.845515,2.031745e+09,15487372,1.739951e+07
1327,0.805003,603.870667,-2374.0,0.0,398.0,2501.0,0.018574,0.043082,4.141681,1787.828979,...,1237.712891,-5228.0,-812.0,0.0,5227.0,-0.016879,0.192274,1.070819e+08,428461,4.157316e+06
1328,1.021250,754.624329,-7348.0,0.0,386.0,6818.0,0.011249,6.800555,0.553224,756.680054,...,744.890503,-3657.0,-481.0,0.0,4037.0,-0.015176,0.576139,5.712796e+08,33820073,3.294503e+07


In [24]:
# finalize_model() 
# - fits the model onto the complete dataset including the test/hold-out sample (30% in this case). 
# - The purpose of this function is to train the model on the complete dataset before it is deployed in production.

final_blended = finalize_model(blended)
print(final_blended)

INFO:logs:Initializing finalize_model()
INFO:logs:finalize_model(estimator=VotingRegressor(estimators=[('et',
                             ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0,
                                                 criterion='mse',
                                                 max_depth=None,
                                                 max_features='auto',
                                                 max_leaf_nodes=None,
                                                 max_samples=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100, n_jobs=-1,
                         

VotingRegressor(estimators=[('et',
                             ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0,
                                                 criterion='mse',
                                                 max_depth=None,
                                                 max_features='auto',
                                                 max_leaf_nodes=None,
                                                 max_samples=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100, n_jobs=-1,
                                                 oob_score=False,
                                  

In [25]:
predict_model(final_blended)

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=VotingRegressor(estimators=[('et',
                             ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0,
                                                 criterion='mse',
                                                 max_depth=None,
                                                 max_features='auto',
                                                 max_leaf_nodes=None,
                                                 max_samples=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100, n_jobs=-1,
                           

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,1.200119e+06,2.681192e+12,1.637434e+06,0.9851,0.3406,0.2873


,sensor_1_mean,sensor_1_std,sensor_1_min,sensor_1_50%,sensor_1_75%,sensor_1_max,sensor_1_skew,sensor_1_kurtosis,sensor_2_mean,sensor_2_std,...,sensor_10_std,sensor_10_min,sensor_10_25%,sensor_10_50%,sensor_10_max,sensor_10_skew,sensor_10_kurtosis,segment_id,time,Label
0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,-1.531141,982.825989,...,554.055237,-3098.0,-348.0,0.0,3592.0,0.043389,0.878872,5.949085e+08,47775708,4.600389e+07
1,2.774204,271.149200,-1487.0,0.0,168.0,1732.0,0.043931,1.781223,0.000000,0.000000,...,588.235718,-4149.0,-367.0,0.0,4052.0,0.052804,1.859576,4.786741e+08,17722981,1.721027e+07
2,-3.869586,155.141571,-656.0,0.0,98.0,748.0,-0.086593,0.321308,-1.642989,464.430237,...,291.042755,-1556.0,-184.0,0.0,1545.0,0.031023,0.812098,1.759210e+09,38195716,3.726920e+07
3,1.258896,507.877563,-4190.0,0.0,303.0,3769.0,0.000385,2.031903,5.067732,826.030701,...,564.066833,-2553.0,-364.0,0.0,2584.0,0.013483,0.312417,2.889141e+08,43673535,4.123369e+07
4,-0.967134,369.767181,-1609.0,0.0,229.0,1919.0,0.013579,0.864417,-1.387227,930.861328,...,903.474915,-4721.0,-561.0,0.0,5130.0,-0.022145,1.884141,1.534910e+09,84390,3.285650e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1325,7.926568,378.016266,-1945.0,0.0,256.0,1636.0,0.001878,0.621614,0.015383,565.777527,...,776.656616,-4215.0,-482.0,0.0,5889.0,0.076811,1.427138,1.357148e+09,47055965,4.190622e+07
1326,0.750104,645.540283,-3164.0,0.0,417.0,2989.0,0.015595,0.574703,5.097498,904.946106,...,1003.427673,-5722.0,-642.0,0.0,5553.0,-0.009897,0.845515,2.031745e+09,15487372,1.640648e+07
1327,0.805003,603.870667,-2374.0,0.0,398.0,2501.0,0.018574,0.043082,4.141681,1787.828979,...,1237.712891,-5228.0,-812.0,0.0,5227.0,-0.016879,0.192274,1.070819e+08,428461,1.526156e+06
1328,1.021250,754.624329,-7348.0,0.0,386.0,6818.0,0.011249,6.800555,0.553224,756.680054,...,744.890503,-3657.0,-481.0,0.0,4037.0,-0.015176,0.576139,5.712796e+08,33820073,3.318909e+07


In [26]:
save_model(final_blended, 'final_blended')

INFO:logs:Initializing save_model()
INFO:logs:save_model(model=VotingRegressor(estimators=[('et',
                             ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0,
                                                 criterion='mse',
                                                 max_depth=None,
                                                 max_features='auto',
                                                 max_leaf_nodes=None,
                                                 max_samples=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100, n_jobs=-1,
                                     

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=False, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=[], target='time',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_strategy='...
                                                            interaction_constraints='',
                                                            learning_rate=0.300000012,
                                                            max_bin=256,
                                                            max_c

In [27]:
# files.download('final_blended.pkl')